In [13]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import pandas as pd
import re
df = pd.read_csv("../../data/external/processed/all_hla.csv")

In [72]:
paths = glob.glob(
    "/home/daqop/mountpoint_snellius/softwares/gibbscluster-2.0/run/all_hla_peptides_clusters_1_15_with_trash_1876625/cores/gibbs.*of*.core"
)
alphabet = list("ARNDCQEGHILKMFPSTWYV")
p_matrices = []
p_matrice_names = []
for path in paths:
    core_f = open(path, "r")
    f_name = path.split("/")[-1].replace(".core", "")
    cluster = int(re.search("(?<=gibbs.)[0-9]*", f_name).group())
    cluster_set = int(re.search("(?<=of)[0-9]*", f_name).group())
    p_matrice_names.append({
        "cluster": cluster,
        "cluster_set": cluster_set
    })
    peptides = []
    for l in core_f:
        peptides.append(list(l.replace("\n","")))
    peptides = np.array(peptides)
    p_matrice = np.array([np.count_nonzero(peptides == alphabet[i], axis=0)/peptides.shape[0] for i in range(len(alphabet))])
    p_matrices.append(p_matrice)
p_matrices = np.stack(p_matrices)

In [90]:
all_klds = []
for i in range(1,5):
    q_matrices = np.stack([p_matrices[p_matrice_names.index(name)] for name in p_matrice_names if name["cluster_set"] == i])
    print(q_matrices.shape)
    for j in range(i):
        p_matrice = [p_matrices[p_matrice_names.index(name)] for name in p_matrice_names if name["cluster"] == j+1 and name["cluster_set"] == i][0]
        for k in range(1, q_matrices.shape[0]+1):
            q_matrice = [p_matrices[p_matrice_names.index(name)] for name in p_matrice_names if name["cluster_set"] == k and name["cluster"] == k]
            # CALCULATE KLD
        # name_i = p_matrice_names.index(p_matrice_name)
        # p_matrice = p_matrices[name_i]
        # q_matrices = 

(1, 20, 9)
(2, 20, 9)
(3, 20, 9)
(4, 20, 9)


In [23]:
# retrieve log odds of matrices and calculate KLD distances:
bins = np.arange(0,0.1,0.001)
f_paths = glob.glob(
    f"/home/daqop/mountpoint_snellius/softwares/gibbscluster-2.0/run/all_hla_peptides_clusters_1_15_with_trash_1876625/matrices/*.mat"
)
p_matrices = []
p_matrice_names = []
for i,f_path in enumerate(f_paths):
    # if i == 0:
    f_name = f_path.split("/")[-1].replace(".mat", "")
    cluster = int(re.search("(?<=gibbs.)[0-9]*", f_name).group())-1
    cluster_set = str(re.search("(?<=of)[0-9]*", f_name).group())
    n = len(set(df.loc[df[f"cluster_set_{cluster_set}"] == float(cluster)]["peptide"].tolist()))
    # else:
    #     break
    # print(f"Filename: {f_name}, cluster: {cluster}, cluster_group: {cluster_set},n = {n}")
    f = open(f_path, "r")
    lines = [l.replace("\n", "") for l in f]
    log_odds = []
    for i,line in enumerate(lines):
        if i <= 1:
            continue
        log_odds.append(line.split()[2:])
    log_odds = np.array(log_odds, dtype=np.float32)
    p = np.exp(log_odds)/20*(np.exp(n/(n+5)))
    p_matrice_names.append(f_name)
    p_matrices.append(p)
p_matrices = np.stack(p_matrices)
print(p_matrices.min())
print(p_matrices.max())

7.860078e-06
247.06195
